In [9]:
# Crunchbase analysis Jupyter notebook.
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style
import pandas
import numpy as np
import sklearn as sk
from pandas import DataFrame as df

style.use('fivethirtyeight')


In [7]:
# CSS files for more aesthetically pleasing inline tables.
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [42]:
# Import Crunchbase 2013 Snapshot CSV files.
cb_acquisitions = df.from_csv('csv/cb_acquisitions.csv').reset_index('id')
cb_degrees = df.from_csv('csv/cb_degrees.csv').reset_index('id')
cb_funding_rounds = df.from_csv('csv/cb_funding_rounds.csv').reset_index('id')
cb_funds = df.from_csv('csv/cb_funds.csv').reset_index('id')
cb_investments = df.from_csv('csv/cb_investments.csv').reset_index('id')
cb_ipos = df.from_csv('csv/cb_ipos.csv').reset_index('id')
cb_milestones = df.from_csv('csv/cb_milestones.csv').reset_index('id')
cb_objects = df.from_csv('csv/cb_objects.csv').reset_index('id')
cb_offices = df.from_csv('csv/cb_offices.csv').reset_index('id')
cb_people = df.from_csv('csv/cb_people.csv').reset_index('id')
cb_relationships = df.from_csv('csv/cb_relationships.csv').reset_index('id')

# Import ODM data from CSVs.
odm_organizations = df.from_csv('csv/odm_organizations.csv')
odm_people = df.from_csv('csv/odm_people.csv')

,type,primary_role,name,crunchbase_url,homepage_domain,homepage_url,profile_image_url,facebook_url,twitter_url,linkedin_url,stock_symbol,location_city,location_region,location_country_code,short_description
crunchbase_uuid,,,,,,,,,,,,,,,
e1393508-30ea-8a36-3f96-dd3226033abd,Organization,company,Wetpaint,https://www.crunchbase.com/organization/wetpai...,wetpaint-inc.com,http://wetpaint-inc.com,https://www.crunchbase.com/organization/wetpai...,NaN,http://twitter.com/BachelrWetpaint,NaN,:,Seattle,Washington,USA,Wetpaint offers an online social publishing pl...
bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Organization,company,Zoho,https://www.crunchbase.com/organization/zoho?u...,zoho.com,https://www.zoho.com/,https://www.crunchbase.com/organization/zoho/p...,http://www.facebook.com/zoho,http://twitter.com/zoho,http://www.linkedin.com/company/zoho-corporati...,:,Pleasanton,California,USA,Run your entire business with Zoho's suite of ...
5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Organization,company,Digg,https://www.crunchbase.com/organization/digg?u...,digg.com,http://www.digg.com,https://www.crunchbase.com/organization/digg/p...,http://www.facebook.com/digg,http://twitter.com/digg,http://www.linkedin.com/company/digg,:,NaN,NaN,NaN,Digg Inc. operates a website that enables its ...
f4d5ab44-058b-298b-ea81-380e6e9a8eec,Organization,investor,Omidyar Network,https://www.crunchbase.com/organization/omidya...,omidyar.com,http://www.omidyar.com,https://www.crunchbase.com/organization/omidya...,http://www.facebook.com/OmidyarNetwork,http://twitter.com/OmidyarNetwork,http://www.linkedin.com/company/22806,:,Redwood City,California,USA,"Pierre Omidyar, the founder of eBay, and his w..."
df662812-7f97-0b43-9d3e-12f64f504fbb,Organization,company,Facebook,https://www.crunchbase.com/organization/facebo...,facebook.com,http://www.facebook.com,https://www.crunchbase.com/organization/facebo...,https://www.facebook.com/,https://twitter.com/facebook,http://www.linkedin.com/company/facebook,NASDAQ:FB,Menlo Park,California,USA,Facebook is an online social networking servic...
